In [0]:
dbutils.fs.mount(
            source = "wasbs://inputd@ajokestorage01.blob.core.windows.net",
            mount_point = "/mnt/raw",
            extra_configs = {"fs.azure.account.key.ajokestorage01.blob.core.windows.net":"<include your key here>"})

Out[2]: True

In [0]:
#Check folder and the data mounted from Azure blob storage
dbutils.fs.ls("/mnt/raw")

Out[4]: [FileInfo(path='dbfs:/mnt/raw/pizza_sales.csv', name='pizza_sales.csv', size=8859953, modificationTime=1716490781000)]

In [0]:
#Create Dataframe
df = spark.read.format("csv").options(header='True', inferSchema= 'True').load('dbfs:/mnt/raw/pizza_sales.csv')

In [0]:
#Show first 5 rows on dataframe
display(df.limit(5))

order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
1,1,hawaiian_m,1,2015-01-01,2024-05-27T11:38:36.000+0000,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
2,2,classic_dlx_m,1,2015-01-01,2024-05-27T11:57:40.000+0000,16.0,16.0,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon",The Classic Deluxe Pizza
3,2,five_cheese_l,1,2015-01-01,2024-05-27T11:57:40.000+0000,18.5,18.5,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic",The Five Cheese Pizza
4,2,ital_supr_l,1,2015-01-01,2024-05-27T11:57:40.000+0000,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",The Italian Supreme Pizza
5,2,mexicana_m,1,2015-01-01,2024-05-27T11:57:40.000+0000,16.0,16.0,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red Onions, Cilantro, Corn, Chipotle Sauce, Garlic",The Mexicana Pizza


In [0]:
#Convert dataframe into View so as to run SQL table
df.createOrReplaceTempView("pizza_sales_analysis")

In [0]:
%sql 
-- SQL in Databricks uses MySQL flavour query method
SELECT * FROM pizza_sales_analysis LIMIT 5

order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
1,1,hawaiian_m,1,2015-01-01,2024-05-27T11:38:36.000+0000,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
2,2,classic_dlx_m,1,2015-01-01,2024-05-27T11:57:40.000+0000,16.0,16.0,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon",The Classic Deluxe Pizza
3,2,five_cheese_l,1,2015-01-01,2024-05-27T11:57:40.000+0000,18.5,18.5,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic",The Five Cheese Pizza
4,2,ital_supr_l,1,2015-01-01,2024-05-27T11:57:40.000+0000,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",The Italian Supreme Pizza
5,2,mexicana_m,1,2015-01-01,2024-05-27T11:57:40.000+0000,16.0,16.0,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red Onions, Cilantro, Corn, Chipotle Sauce, Garlic",The Mexicana Pizza


In [0]:
%sql 
--CREATE AGGREGATE TABLE FOR SPECIFIC REPORTING
SELECT 
count(DISTINCT order_id) Order_id, 
sum(quantity) Quantity, 
date_format(order_date,'MMM') Month, 
date_format(order_date,'EEE') Day_Of_Week, 
hour(order_time) Order_Time, 
sum(unit_price) Unit_Price, 
sum(total_price) Total_Sales, 
pizza_size, 
pizza_category,
pizza_name
FROM pizza_sales_analysis
GROUP BY 3,4,5,8,9,10
LIMIT 7

Order_id,Quantity,Month,Day_Of_Week,Order_Time,Unit_Price,Total_Sales,pizza_size,pizza_category,pizza_name
2,2,Jun,Mon,19,33.0,33.0,M,Supreme,The Pepper Salami Pizza
3,3,Feb,Thu,12,55.5,55.5,L,Veggie,The Five Cheese Pizza
1,1,Feb,Tue,13,12.25,12.25,S,Supreme,The Sicilian Pizza
1,1,May,Mon,11,12.5,12.5,S,Supreme,The Spinach Supreme Pizza
1,1,Jun,Wed,21,20.25,20.25,L,Veggie,The Mexicana Pizza
2,2,May,Sat,18,40.5,40.5,L,Veggie,The Mediterranean Pizza
3,3,May,Sun,16,44.25,44.25,M,Veggie,The Four Cheese Pizza


In [0]:
%sql 
--CREATE TABLE FROM THE ABOVE QUERY
CREATE TABLE KPI_Table AS
SELECT 
count(DISTINCT order_id) Order_id, 
sum(quantity) Quantity, 
date_format(order_date,'MMM') Month, 
date_format(order_date,'EEE') Day_Of_Week, 
hour(order_time) Order_Time, 
sum(unit_price) Unit_Price, 
sum(total_price) Total_Sales, 
pizza_size, 
pizza_category,
pizza_name
FROM pizza_sales_analysis
GROUP BY 3,4,5,8,9,10


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM KPI_Table LIMIT 10

Order_id,Quantity,Month,Day_Of_Week,Order_Time,Unit_Price,Total_Sales,pizza_size,pizza_category,pizza_name
2,2,Jun,Mon,19,33.0,33.0,M,Supreme,The Pepper Salami Pizza
3,3,Feb,Thu,12,55.5,55.5,L,Veggie,The Five Cheese Pizza
1,1,Feb,Tue,13,12.25,12.25,S,Supreme,The Sicilian Pizza
1,1,May,Mon,11,12.5,12.5,S,Supreme,The Spinach Supreme Pizza
1,1,Jun,Wed,21,20.25,20.25,L,Veggie,The Mexicana Pizza
2,2,May,Sat,18,40.5,40.5,L,Veggie,The Mediterranean Pizza
3,3,May,Sun,16,44.25,44.25,M,Veggie,The Four Cheese Pizza
1,1,May,Mon,12,16.0,16.0,M,Classic,The Classic Deluxe Pizza
3,4,Jan,Tue,13,60.75,81.0,L,Veggie,The Spinach and Feta Pizza
2,2,Apr,Sun,17,26.5,26.5,M,Classic,The Hawaiian Pizza
